# 1. Speech Recognition

In [ ]:
pip install SpeechRecognition

In [ ]:
import speech_recognition as sr

In [ ]:
from os import path
AUDIO_FILE = path.join(path.dirname(path.realpath("test.wav")), "test.wav")

In [ ]:
r = sr.Recognizer()
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)

In [ ]:
try:
    # for testing purposes, we're just using the default API key
    # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    # instead of `r.recognize_google(audio)`
    print(r.recognize_google(audio))
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))

# 2. Rule-Based Named-Entity-Recognition

## 2.2. RBNER - LAGE



In [ ]:
import pandas as pd
import spacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.pipeline import EntityRuler
from spacy import displacy
from spacy.lang.de import German
from spacy.lang.en import English

In [ ]:
nlp_german = spacy.load("de_core_news_sm", exclude="ner") 
nlp_german = German()

In [ ]:
german_ruler = nlp_german.add_pipe("entity_ruler", config={"validate":True})

In [ ]:
with open('XXX_LAGE.txt', 'r') as file:
    lage = file.read().replace('\n',' ')

In [ ]:
with open('parteien.txt', 'r') as file:
    parteien = file.read()
    
parteien_list = parteien.split("\n")

parteien_list

In [ ]:
with open('politiker_innen.txt', 'r') as file:
    politiker_innen = file.read()
    
politiker_innen_list = politiker_innen.split("\n")    

politiker_innen_list

In [ ]:
with open('parteien.txt', 'r') as file:
    parteien = file.read()
    
parteien_list = parteien.split("\n")

parteien_list

In [ ]:
parteien_patterns = [
    {"label": "PARTEI", "pattern": str(partei)} for partei in list(nlp_german.pipe(parteien_list))
]
politiker_innen_patterns = [
    {"label": "POLITIKER_IN", "pattern": str(politiker_in)} for politker_in in list(nlp_german.pipe(politiker_innen_list))
]

# Schritt 4.3: Zusammenführen der Patterns zu einer Liste
patterns = parteien_patterns + politiker_innen_patterns
patterns

In [ ]:
german_ruler.add_patterns(patterns)

In [ ]:
bundestag_protokoll_text_processed = nlp_german(lage)

## 2.1. RBNER - SPIEGEL

In [ ]:
nlp_german = spacy.load("de_core_news_sm", exclude="ner") 
nlp_german = German()

In [ ]:
german_ruler = nlp_german.add_pipe("entity_ruler", config={"validate":True})

In [ ]:
with open('XXX_SPIEGEL.txt', 'r') as file:
    spiegel = file.read().replace('\n',' ')

In [ ]:
german_ruler.add_patterns(patterns)

In [ ]:
bundestag_protokoll_text_processed = nlp_german(spiegel)

## 2.1. RBNER - FAZ

In [ ]:
nlp_german = spacy.load("de_core_news_sm", exclude="ner") 
nlp_german = German()

In [ ]:
german_ruler = nlp_german.add_pipe("entity_ruler", config={"validate":True})

In [ ]:
with open('XXX_FAZ.txt', 'r') as file:
    faz = file.read().replace('\n',' ')

In [ ]:
german_ruler.add_patterns(patterns)

In [ ]:
bundestag_protokoll_text_processed = nlp_german(faz)

# 3. Sentiment-Analyse

In [ ]:
!pip install transformers

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import matthews_corrcoef, f1_score, recall_score, precision_score, make_scorer
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from statistics import mean
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from sklearn.metrics import matthews_corrcoef, f1_score, recall_score, precision_recall_fscore_support, precision_score, make_scorer

In [ ]:
# wir verwenden den AutoTokenizer, da er automatisch erkennt, welcher Tokenizer
# benutzt werden muss und für welches Modell, z.B. BERT/T5.
# Grund: Modelle müssen teilweise unterschiedlich tokenisiert werden
from transformers import AutoTokenizer 

# als "Checkpoint" wird das Transformer-Model, das wir nutzen, bezeichnet
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# hier laden wir den Tokenizer unseres Checkpoints herunter und speichern ihn
# in einer Variable
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
from pprint import pprint

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

# nun wenden wir den Tokenizer auf den Rohtext an. Das Ergebnis wird in einer 
# Variable gespeichert und im Anschluss für das Model verwendet.

# padding = True => BERT kann Sätze mit einer Länge von maximal 512 Tokens verarbeiten.
# Wenn padding = True werden kürzere Sequenzen auf die maximale Länge im Korpus (<=512 Token) gestreckt bzw. aufgefüllt

# truncation = True => enthält eine Sequenz mehr Token als zulässig sind, werden die überschüssigen Tokens entfernt

# return_tensors = "pt" => spezifiziert den Tensor-Typ, den wir zurückerhalten: 
# pt = PyTorch, tf = TensorFlow, np = NumPy
model_inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
# So sieht der Model-Input aus...
pprint(model_inputs)

In [ ]:
# 2 => zwei Input-Sätze
# 16 => Sequence Length, also die maximale Tokenlänge
# 768 => Anzahl der Dimensionen des Vektors
pprint(outputs.last_hidden_state.shape)

In [ ]:
from transformers import AutoModelForSequenceClassification

# Indem wir das AutoModelForSequenceClassification verwenden, können wir nun auf
# den Head zugreifen
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**model_inputs)
pprint(outputs)

In [ ]:
# Der Head-Output enthält sogenannte logits
# 2 => Anzahl Sätze
# 2 => Anzahl Labels
pprint(outputs.logits.shape)

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
pprint(predictions)

In [ ]:
# Mit diesem Befehl können wir auf Model Config zugreifen und herausfinden, welche
# Indexposition welchem Label entspricht
model.config.id2label

evtl vereinfachen als Pipe
Evaluation
Tabellarische Übersicht möglich als Output (Freq etc)